In [1]:
import pandas as pd
from datetime import datetime, timedelta
import re
from difflib import get_close_matches
import warnings
warnings.filterwarnings('ignore')


countries_df = pd.read_excel("My port\My port\Countries IDs.xlsx")
codes_df = pd.read_excel("My port\My port\Ports_ceties_codes_and_IDs.xlsx")
my_port_aug_file = pd.ExcelFile("My port\My port\MyPoert AUG sheet1.xlsx")



#----------matching variations of 'Trip' and 'Price Offer'-----------
trip_pattern = re.compile(r'Trip\s*', flags=re.IGNORECASE)
price_offer_pattern = re.compile(r'Price\s*Offer|Price\s*', flags=re.IGNORECASE)



def find_approximate_match(word, choices):
    matches = get_close_matches(word, choices, n=1, cutoff=0.1)
    return matches[0] if matches else None



combined_results = pd.DataFrame()




#-----------iterate over each sheet and process the data--------------------
for sheet_name in my_port_aug_file.sheet_names:
    my_port_aug_df = pd.read_excel(my_port_aug_file, sheet_name=sheet_name)
    my_port_aug_df = my_port_aug_df.iloc[:, :2] 
    my_port_aug_df = my_port_aug_df.dropna()

    trip_column = [col for col in my_port_aug_df.columns if trip_pattern.search(col)]
    price_offer_column = [col for col in my_port_aug_df.columns if price_offer_pattern.search(col)]
    
    if len(trip_column) == 1:
        trip_column = trip_column[0]
    else:
        raise ValueError("Trip column not found or multiple Trip columns present")

    if len(price_offer_column) == 1:
        price_offer_column = price_offer_column[0]
    else:
        raise ValueError("Price Offer column not found or multiple Price Offer columns present")

    my_port_aug_df[trip_column] = my_port_aug_df[trip_column].str.strip()

    
    #---------loop in rows------------
    for index, row in my_port_aug_df.iterrows():
        trip_parts = row[trip_column].split("TO")
        if len(trip_parts) != 2:
            continue

            
        from_city = find_approximate_match(trip_parts[0].split("FROM")[-1].strip(), codes_df['Port Name'])
        to_city = find_approximate_match(trip_parts[1].strip(), codes_df['Port Name'])

        
        if from_city and to_city:
            from_code_id = codes_df.loc[codes_df['Port Name'] == from_city, ['Code', 'ID']].values
            to_code_id = codes_df.loc[codes_df['Port Name'] == to_city, ['Code', 'ID']].values

            my_port_aug_df.at[index, trip_column] = f"FROM {from_city} TO {to_city}"

            my_port_aug_df.at[index, 'From Code'] = from_code_id[0][0]
            my_port_aug_df.at[index, 'From ID'] = from_code_id[0][1]
            my_port_aug_df.at[index, 'To Code'] = to_code_id[0][0]
            my_port_aug_df.at[index, 'To ID'] = to_code_id[0][1]
            
            if "ONE WAY" in row[price_offer_column].upper():
                my_port_aug_df.at[index, 'trip_type'] = 'O'
            elif "ROUND" in row[price_offer_column].upper():
                my_port_aug_df.at[index, 'trip_type'] = 'R'
                
            my_port_aug_df['URL'] = my_port_aug_df['From Code'] + '-' + my_port_aug_df['To Code'] + '-' + my_port_aug_df['trip_type']
            my_port_aug_df['URL'] = my_port_aug_df['URL'].str.lower()    

            my_port_aug_df.at[index, 'publish_date'] = datetime.now().strftime('%Y-%m-%d')
            publish_date = datetime.now()
            my_port_aug_df.at[index, 'expir_date'] = (publish_date + timedelta(days=35)).strftime('%Y-%m-%d')

        else:
            print(f"Approximate match not found for row {index}")

            
            
            
    my_port_aug_df['From_Port'] = my_port_aug_df[trip_column].str.extract(r'FROM\s+(.*?)\s+TO', flags=re.IGNORECASE)
    my_port_aug_df['To_Port'] = my_port_aug_df[trip_column].str.extract(r'TO\s+(.*?)\s+', flags=re.IGNORECASE)

    combined_results = combined_results.append(my_port_aug_df[['From ID', 'To ID', 'trip_type', 'URL', 'publish_date', 'expir_date']], ignore_index=True)

    
    
    
    
    
    
combined_results.to_excel("offers_tamplet.xlsx", index=False)


Approximate match not found for row 1
Approximate match not found for row 2
Approximate match not found for row 10
Approximate match not found for row 12


In [3]:


trip_pattern = re.compile(r'Trip\s*', flags=re.IGNORECASE)
price_offer_pattern = re.compile(r'Price\s*Offer|Price\s*', flags=re.IGNORECASE)



def find_approximate_match(word, choices):
    matches = get_close_matches(word, choices, n=1, cutoff=0.8)
    return matches[0] if matches else None


combined_results = pd.DataFrame()


for sheet_name in my_port_aug_file.sheet_names:
    my_port_aug_df = pd.read_excel(my_port_aug_file, sheet_name=sheet_name)
    my_port_aug_df = my_port_aug_df.iloc[:, :2] 
    my_port_aug_df = my_port_aug_df.dropna()

    my_port_aug_df['country_id'] = None

    trip_column = [col for col in my_port_aug_df.columns if trip_pattern.search(col)]
    price_offer_column = [col for col in my_port_aug_df.columns if price_offer_pattern.search(col)]
    
    if len(trip_column) == 1:
        trip_column = trip_column[0]
    else:
        raise ValueError("Trip column not found or multiple Trip columns present")

    if len(price_offer_column) == 1:
        price_offer_column = price_offer_column[0]
    else:
        raise ValueError("Price Offer column not found or multiple Price Offer columns present")

    my_port_aug_df[trip_column] = my_port_aug_df[trip_column].str.strip()

    for index, row in my_port_aug_df.iterrows():
        trip_parts = row[trip_column].split("TO")
        if len(trip_parts) != 2:
            continue  

        from_city = find_approximate_match(trip_parts[0].split("FROM")[-1].strip(), codes_df['Port Name'])
        to_city = find_approximate_match(trip_parts[1].strip(), codes_df['Port Name'])

        if from_city and to_city:
            from_code_id = codes_df.loc[codes_df['Port Name'] == from_city, ['Code', 'ID']].values
            to_code_id = codes_df.loc[codes_df['Port Name'] == to_city, ['Code', 'ID']].values

            my_port_aug_df.at[index, trip_column] = f"FROM {from_city} TO {to_city}"

            my_port_aug_df.at[index, 'From Code'] = from_code_id[0][0]
            my_port_aug_df.at[index, 'From ID'] = from_code_id[0][1]
            my_port_aug_df.at[index, 'To Code'] = to_code_id[0][0]
            my_port_aug_df.at[index, 'To ID'] = to_code_id[0][1]
            
            if "ONE WAY" in row[price_offer_column].upper():
                my_port_aug_df.at[index, 'trip_type'] = 'O'
            elif "ROUND" in row[price_offer_column].upper():
                my_port_aug_df.at[index, 'trip_type'] = 'R'
                
            my_port_aug_df['URL'] = my_port_aug_df['From Code'] + '-' + my_port_aug_df['To Code'] + '-' + my_port_aug_df['trip_type']
            my_port_aug_df['URL'] = my_port_aug_df['URL'].str.lower()    

            my_port_aug_df.at[index, 'publish_date'] = datetime.now().strftime('%Y-%m-%d')
            publish_date = datetime.now()
            my_port_aug_df.at[index, 'expir_date'] = (publish_date + timedelta(days=35)).strftime('%Y-%m-%d')
            
            country_name = sheet_name
            
            country_id_series = countries_df.loc[countries_df['Country'] == country_name, 'ID']
            if not country_id_series.empty:
                country_id = country_id_series.iloc[0]

                my_port_aug_df.at[index, 'country_id'] = country_id
            else:
                print(f"No matching country found for sheet '{sheet_name}'")

        else:
            print(f"Approximate match not found for row {index}")

    combined_results = combined_results.append(my_port_aug_df[['URL', 'country_id']], ignore_index=True)

combined_results.to_excel("offer_content.xlsx", index=False)


Approximate match not found for row 1
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
No matching country found for sheet 'Paramaribo'
Approximate match not found for row 2
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found for sheet 'Majuro'
No matching country found